In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer
import pickle

In [26]:
# Load the dataset
df = pd.read_excel("E://AI//Fitness app data.xlsx")

In [27]:
df.head(12)

,Goal,Type,Exercise Type,Exercise Plan,Cardio Minutes,Strength Training,Water Intake (Liters),Sleep Hours,Diet Plan,Protein (g),Carbs (g),Fats (g),Daily Routine
0,Lean,Cut,"Cardio, Light Strength","30 mins Cardio, 3x/week Strength",30,3x/week,2.0,8,"1800 kcal, 90g Protein, 200g Carbs",90,200,50,"Morning: Cardio, Afternoon: Work, Evening: Str..."
1,Lean,Athletic,"Cardio, Strength","45 mins Cardio, 4x/week Strength",40,4x/week,2.5,8,"1900 kcal, 100g Protein, 220g Carbs",100,220,60,"Morning: Cardio, Afternoon: Work, Evening: Str..."
2,Bulky,Muscular,"Strength, Training","5x/week Heavy Lifting, Focus on Power",Zero,5x/week,3.0,8,"2500 kcal, 150g Protein, 300g Carbs",150,300,80,"Morning: Strength, Afternoon: Rest, Evening: C..."
3,Bulky,Ripped,"Strength, Cardio","4x/week Strength, 2x/week Cardio",30,4x/week,3.5,8,"2700 kcal, 180g Protein, 350g Carbs",180,350,90,"Morning: Strength, Afternoon: Cardio, Evening:..."
4,Jacked,Muscular,"HIIT, Strength","3x/week Strength, 3x/week HIIT",45,3x/week,4.0,7,"3000 kcal, 200g Protein, 350g Carbs",200,350,100,"Morning: Strength, Afternoon: Rest, Evening: C..."
5,Jacked,Ripped,"HIIT, Strength, Power","3x/week Strength, 3x/week HIIT",60,3x/week,2.5,7,"3200 kcal, 250g Protein, 400g Carbs",250,400,120,"Morning: Strength, Afternoon: Rest, Evening: C..."
6,Toned,Athletic,"Cardio, Strength","3x/week Strength, 2x/week Cardio",30,3x/week,2.5,8,"2200 kcal, 120g Protein, 250g Carbs",120,250,70,"Morning: Cardio, Afternoon: Work, Evening: Str..."
7,Toned,Lean,"Cardio, Strength","4x/week Strength, 2x/week Cardio",30,4x/week,2.5,8,"2000 kcal, 110g Protein, 180g Carbs",110,180,60,"Morning: Cardio, Afternoon: Work, Evening: Str..."
8,Athletic,Muscular,Mixed Routine,"2x/week Strength, 3x/week Cardio",30,2x/week,2.5,8,"2300 kcal, 130g Protein, 270g Carbs",130,270,75,"Morning: Cardio, Afternoon: Work, Evening: Str..."
9,Athletic,Ripped,"Strength, Cardio","3x/week Strength, 3x/week Cardio",40,3x/week,3.0,8,"2400 kcal, 140g Protein, 280g Carbs",140,280,80,"Morning: Strength, Afternoon: Cardio, Evening:..."


In [28]:
# Preprocessing: Encoding the Alphabetic data into numeric data so it could work on
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column].astype(str))
    label_encoders[column] = le

In [29]:
# Handle missing data values in the dataset using KNN Imputer
imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [30]:
# Features and labels (The last of the column would be refer as labels(output) and the rest of it would be features(input))
features = df_imputed.iloc[:, :-1]  # All columns except the last one
labels = df_imputed.iloc[:, -1]  # The last column is the label

In [31]:
df.head(12)

,Goal,Type,Exercise Type,Exercise Plan,Cardio Minutes,Strength Training,Water Intake (Liters),Sleep Hours,Diet Plan,Protein (g),Carbs (g),Fats (g),Daily Routine
0,3,1,0,1,0,1,2.0,8,0,90,200,50,0
1,3,0,1,5,1,2,2.5,8,1,100,220,60,0
2,1,3,6,7,4,3,3.0,8,6,150,300,80,2
3,1,4,5,6,0,2,3.5,8,7,180,350,90,1
4,2,3,2,4,2,1,4.0,7,8,200,350,100,2
5,2,4,3,4,3,1,2.5,7,9,250,400,120,2
6,5,0,1,2,0,1,2.5,8,3,120,250,70,0
7,5,2,1,6,0,2,2.5,8,2,110,180,60,0
8,0,3,4,0,0,0,2.5,8,4,130,270,75,0
9,0,4,5,3,1,1,3.0,8,5,140,280,80,1


In [32]:
# Balance the dataset using SMOTE {to balance an imbalanced dataset by creating synthetic samples for the minority class(lables & features)}
smote = SMOTE(random_state=42, k_neighbors=1)  # Adjusted k_neighbors
features, labels = smote.fit_resample(features, labels)

In [33]:
# Verify the new class distribution
print(labels.value_counts())

Daily Routine
0.0    7
2.0    7
1.0    7
Name: count, dtype: int64


In [34]:
# Split the data into training and testing sets 20% of data would be used for testing
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [35]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [36]:
# Model 1: KNN with hyperparameter tuning (using the hyperparameter to get the best model best perfromance)
knn_params = {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5)
knn_grid.fit(X_train_scaled, y_train)
knn_best = knn_grid.best_estimator_

C:\Users\uuu\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


In [37]:
# Model 2: Logistic Regression with hyperparameter tuning (using the hyperparameter to get the best model best perfromance)
lr_params = {'C': [0.1, 1, 10], 'solver': ['liblinear']}
lr_grid = GridSearchCV(LogisticRegression(), lr_params, cv=5)
lr_grid.fit(X_train_scaled, y_train)
lr_best = lr_grid.best_estimator_

C:\Users\uuu\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


In [38]:
# Model 3: SVM with hyperparameter tuning (using the hyperparameter to get the best model best perfromance)
svm_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
svm_grid = GridSearchCV(SVC(), svm_params, cv=5)
svm_grid.fit(X_train_scaled, y_train)
svm_best = svm_grid.best_estimator_

C:\Users\uuu\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


In [39]:
# Evaluate models (Getting the values of test train split and accuracy plus matrix of the models)
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return accuracy, cm

In [40]:
# KNN Evaluation
knn_accuracy, knn_cm = evaluate_model(knn_best, X_test_scaled, y_test)
print("KNN Accuracy:", knn_accuracy)
print("KNN Confusion Matrix:\n", knn_cm)

KNN Accuracy: 0.8
KNN Confusion Matrix:
 [[2 1 0]
 [0 1 0]
 [0 0 1]]


In [41]:
# Logistic Regression Evaluation
lr_accuracy, lr_cm = evaluate_model(lr_best, X_test_scaled, y_test)
print("Logistic Regression Accuracy:", lr_accuracy)
print("Logistic Regression Confusion Matrix:\n", lr_cm)

Logistic Regression Accuracy: 1.0
Logistic Regression Confusion Matrix:
 [[3 0 0]
 [0 1 0]
 [0 0 1]]


In [42]:
# SVM Evaluation
svm_accuracy, svm_cm = evaluate_model(svm_best, X_test_scaled, y_test)
print("SVM Accuracy:", svm_accuracy)
print("SVM Confusion Matrix:\n", svm_cm)

SVM Accuracy: 0.8
SVM Confusion Matrix:
 [[2 1 0]
 [0 1 0]
 [0 0 1]]


In [43]:
# Compare models and finding the best one
model_accuracies = {
    "KNN": knn_accuracy,
    "Logistic Regression": lr_accuracy,
    "SVM": svm_accuracy,
}
best_model_name = max(model_accuracies, key=model_accuracies.get)
best_model_accuracy = model_accuracies[best_model_name]
print(f"The best model is {best_model_name} with an accuracy of {best_model_accuracy:.4f}.")

The best model is Logistic Regression with an accuracy of 1.0000.


In [44]:
# Saving models to use them
pickle.dump(knn_best, open("knn_model.pkl", "wb"))
pickle.dump(lr_best, open("lr_model.pkl", "wb"))
pickle.dump(svm_best, open("svm_model.pkl", "wb"))

In [45]:
# Fitness Plan Dictionary
fitness_plans = {
    'Lean': {
        'Goal': 'Lean',
        'Exercise Type': 'Cardio',
        'Exercise Plan': '30 mins Cardio, 3x/week Strength',
        'Cardio Minutes': 30,
        'Strength Training': '3x/week',
        'Water Intake (Liters)': 2,
        'Sleep Hours': 8,
        'Diet Plan': '1800 kcal, 90g Protein, 200g Carbs',
        'Protein (g)': 90,
        'Carbs (g)': 200,
        'Fats (g)': 50,
        'Recommend Fitness Routine': 'Morning: Cardio, Afternoon: Work, Evening: Stretching'
    },
    'Athletic': {
        'Goal': 'Athletic',
        'Exercise Type': 'Cardio',
        'Exercise Plan': '45 mins Cardio, 4x/week Strength',
        'Cardio Minutes': 40,
        'Strength Training': '4x/week',
        'Water Intake (Liters)': 2.5,
        'Sleep Hours': 8,
        'Diet Plan': '1900 kcal, 100g Protein, 220g Carbs',
        'Protein (g)': 100,
        'Carbs (g)': 220,
        'Fats (g)': 60,
        'Recommend Fitness Routine': 'Morning: Cardio, Afternoon: Work, Evening: Stretching'
    },
    'Bulky': {
        'Goal': 'Bulky',
        'Exercise Type': 'Strength',
        'Exercise Plan': '5x/week Heavy Lifting, Focus on Power',
        'Cardio Minutes': 0,
        'Strength Training': '5x/week',
        'Water Intake (Liters)': 3,
        'Sleep Hours': 8,
        'Diet Plan': '2500 kcal, 150g Protein, 300g Carbs',
        'Protein (g)': 150,
        'Carbs (g)': 300,
        'Fats (g)': 80,
        'Recommend Fitness Routine': 'Morning: Strength, Afternoon: Rest, Evening: Cardio'
    },
    'Ripped': {
        'Goal': 'Ripped',
        'Exercise Type': 'HIIT',
        'Exercise Plan': '4x/week Strength, 2x/week Cardio',
        'Cardio Minutes': 30,
        'Strength Training': '4x/week',
        'Water Intake (Liters)': 3,
        'Sleep Hours': 8,
        'Diet Plan': '2700 kcal, 180g Protein, 350g Carbs',
        'Protein (g)': 180,
        'Carbs (g)': 350,
        'Fats (g)': 90,
        'Recommend Fitness Routine': 'Morning: Strength, Afternoon: Cardio, Evening: Rest'
    },
    'Jacked': {
        'Goal': 'Jacked',
        'Exercise Type': 'HIIT',
        'Exercise Plan': '3x/week Strength, 3x/week HIIT',
        'Cardio Minutes': 45,
        'Strength Training': '3x/week',
        'Water Intake (Liters)': 4,
        'Sleep Hours': 7,
        'Diet Plan': '3000 kcal, 200g Protein, 350g Carbs',
        'Protein (g)': 200,
        'Carbs (g)': 350,
        'Fats (g)': 100,
        'Recommend Fitness Routine': 'Morning: Strength, Afternoon: Rest, Evening: Cardio'
    },
    'Mixed Routine': {
        'Goal': 'Mixed Routine',
        'Exercise Type': 'Strength, Cardio',
        'Exercise Plan': '2x/week Strength, 3x/week Cardio',
        'Cardio Minutes': 30,
        'Strength Training': '2x/week',
        'Water Intake (Liters)': 2.5,
        'Sleep Hours': 8,
        'Diet Plan': '2300 kcal, 130g Protein, 270g Carbs',
        'Protein (g)': 130,
        'Carbs (g)': 270,
        'Fats (g)': 75,
        'Recommend Fitness Routine': 'Morning: Cardio, Afternoon: Work, Evening: Stretching'
    },
    'Cut': {
        'Goal': 'Cut',
        'Exercise Type': 'Cardio, Strength',
        'Exercise Plan': '3x/week Strength, 2x/week Cardio',
        'Cardio Minutes': 30,
        'Strength Training': '3x/week',
        'Water Intake (Liters)': 2.5,
        'Sleep Hours': 8,
        'Diet Plan': '2200 kcal, 120g Protein, 250g Carbs',
        'Protein (g)': 120,
        'Carbs (g)': 150,
        'Fats (g)': 70,
        'Recommend Fitness Routine': 'Morning: Cardio, Afternoon: Work, Evening: Stretching'
    }
}

In [46]:
# Function to get fitness plan based on user goal
def get_fitness_plan(goal):
    return fitness_plans.get(goal, "Goal not found. Please choose a valid goal.")

In [47]:
# Main function to get user input and display the fitness plan
def main():
    user_goal = input("Enter your goal (e.g., Lean, Athletic, Bulky, Ripped, Jacked, Mixed Routine): ")
    fitness_plan = get_fitness_plan(user_goal)
    print(f"Recommended Fitness Plan: {fitness_plan}")

In [48]:
if __name__ == "__main__":
    main()

Enter your goal (e.g., Lean, Athletic, Bulky, Ripped, Jacked, Mixed Routine):  Bulky


Recommended Fitness Plan: {'Goal': 'Bulky', 'Exercise Type': 'Strength', 'Exercise Plan': '5x/week Heavy Lifting, Focus on Power', 'Cardio Minutes': 0, 'Strength Training': '5x/week', 'Water Intake (Liters)': 3, 'Sleep Hours': 8, 'Diet Plan': '2500 kcal, 150g Protein, 300g Carbs', 'Protein (g)': 150, 'Carbs (g)': 300, 'Fats (g)': 80, 'Recommend Fitness Routine': 'Morning: Strength, Afternoon: Rest, Evening: Cardio'}
